In [1]:
import numpy as np
import pandas as pd
import os
import random
from shutil import copyfile
#import pydicom as dicom
import cv2
from pathlib import Path

# Paths Definition

Careful, the dataset loader separates by whitespaces so in the file names there cannot be any white spaces

### Paths in Ubuntu

In [3]:
SIMM_COVID_DETECTION_PATH = "/media/usuario/WIN10/COVID19/images/SIIM-Detection/siim-covid19-detection"
PAPER_IMAGES_PATH = "/media/usuario/WIN10/COVID19/images/PaperOriginal/"
TEST_PATH = "/media/usuario/WIN10/COVID19/images/test"

### Paths in MAC

In [4]:
BASE_PATH = '/Volumes/Alvaro HD/UPM/Corpus TFM/COVID'

In [5]:
HACKATHON_PATH = os.path.join(BASE_PATH,'hackathon')
SIIM_DETECTION_PATH = os.path.join(BASE_PATH,'siim-covid19-detection')
PAPER_IMAGES_PATH = os.path.join(BASE_PATH,'CroppedSegmentedImages')
COVID_QU_EX_PATH = os.path.join(BASE_PATH,'Masked-COVID-QU-Ex-dataset')

In [17]:
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
mapping = {0:'normal',1:'COVID-19',2:'pneumonia'}

# Reading from paper files

Prepared to work in Kaggle

In [18]:
for j,directory in enumerate(os.listdir(PAPER_IMAGES_PATH)):
    if not directory.startswith('.') and not directory=='CRXNIH':
        for i,file in enumerate(os.listdir(os.path.join(PAPER_IMAGES_PATH,directory))):
            try:
                cat = int(file.split('__')[1])
            except Exception as e:
                cat = int(file.split('_')[1])
            finally:
                count[mapping[cat]] += 1
                entry = [int(count[mapping[cat]]),'paperoriginal'+ '/'+ 'CroppedSegmentedImages' + '/'+ directory + '/' + file,mapping[cat]]
                filename_label[mapping[cat]].append(entry)
print(count)

{'normal': 60594, 'pneumonia': 22746, 'COVID-19': 8014}


# Reading from SIIM-COVID-Detection

In [20]:
image_level_df = pd.read_csv(os.path.join(SIIM_DETECTION_PATH,'train_image_level.csv'))
study_level_df = pd.read_csv(os.path.join(SIIM_DETECTION_PATH,'train_study_level.csv'))

In [8]:
image_level_df.head(2)

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed


In [9]:
study_level_df.head(2)

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1


In [21]:
study_level_df['StudyInstanceUID'] = study_level_df['id'].apply(lambda x: x.replace('_study',''))
study_level_df.drop('id',axis=1,inplace=True)
train_df = image_level_df.merge(study_level_df,on='StudyInstanceUID')
train_df['id'] = train_df['id'].apply(lambda x: x.replace('_image',''))
train_df.sort_values(by='StudyInstanceUID',ascending=True)

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
2651,65761e66de9f,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852,0,1,0,0
2152,51759b5579bc,NaN,none 1 0 0 1 1,000c9c05fd14,0,0,0,1
6098,f6293b1c49e2,NaN,none 1 0 0 1 1,00292f8c37bd,1,0,0,0
1276,3019399c31f4,NaN,none 1 0 0 1 1,005057b3f880,1,0,0,0
4717,bb4b1da810f3,"[{'x': 812.54698, 'y': 1376.41291, 'width': 62...",opacity 1 812.54698 1376.41291 1435.14793 1806...,0051d9b12e72,0,0,0,1
...,...,...,...,...,...,...,...,...
779,1c96d9b08487,NaN,none 1 0 0 1 1,ffcb4630f46f,0,1,0,0
5768,e6215d0188e5,"[{'x': 364.93056, 'y': 870.04017, 'width': 731...",opacity 1 364.93056 870.04017 1096.13908 2053....,ffe4d6e8fbb0,0,1,0,0
3245,7d27b1bb3987,"[{'x': 28.48292, 'y': 828.48474, 'width': 1116...",opacity 1 28.48292 828.48474 1145.01081 2296.7...,ffe94fcb14fa,0,1,0,0
2170,52478e480a15,"[{'x': 425.81211, 'y': 424.86147, 'width': 528...",opacity 1 425.81211 424.86147 953.95118 1579.3...,ffebf1ef4a9c,0,1,0,0


In [23]:
train_df['StudyInstanceUID'].value_counts()

0fd2db233deb    9
a7335b2f9815    7
8943d1d85097    6
a4e94133d95a    5
a0254bf8a96e    5
               ..
e057d156dc1b    1
f556a329c873    1
206b9724427d    1
dbf24d77289c    1
7d82d53204b8    1
Name: StudyInstanceUID, Length: 6054, dtype: int64

1       NaN
6       NaN
25      NaN
26      NaN
27      NaN
       ... 
6322    NaN
6324    NaN
6325    NaN
6329    NaN
6330    NaN
Name: boxes, Length: 1736, dtype: object

In [13]:
paths = []
for id in train_df['id']:
    paths.append(os.path.join(SIIM_DETECTION_PATH,'train',id + '.jpg'))

In [14]:
labels = []
for id in train_df['id']:
    if (train_df[train_df['id']==id]['Negative for Pneumonia'].values == 1):
        labels.append('normal')
    elif (train_df[train_df['id']==id]['Typical Appearance'].values == 1):
        labels.append('COVID-19')
    elif (train_df[train_df['id']==id]['Indeterminate Appearance'].values == 1):
        labels.append('COVID-19')
    elif (train_df[train_df['id']==id]['Atypical Appearance'].values == 1):
        labels.append('COVID-19')
    else:
        raise Exception

In [15]:
for j,file in enumerate(paths):
    count[labels[j]] += 1
    entry = [int(count[labels[j]]),file,labels[j]]
    filename_label[labels[j]].append(entry)
print(count)

{'normal': 120242, 'pneumonia': 24114, 'COVID-19': 12612}


# Reading from Masked-COVID-QU-Ex-Dataset

In [19]:
for i,parent_directory in enumerate(os.listdir(COVID_QU_EX_PATH)):
    if not parent_directory.startswith('.'):
        full_parent_directory = os.path.join(COVID_QU_EX_PATH,parent_directory)
        for j,category in enumerate(os.listdir(full_parent_directory)):
            if not category.startswith('.'):
                full_category_directory = os.path.join(full_parent_directory,category)
                for k,img_dir in enumerate(os.listdir(full_category_directory)):
                    if not img_dir.startswith('.'):
                        img_dir = img_dir.replace(' ','_')
                        count[category] += 1
                        entry = [int(count[category]) ,os.path.join('maskedcovidquexdataset',parent_directory,category,img_dir)
                        , category]
                        filename_label[category].append(entry)
print(count)

{'normal': 71295, 'pneumonia': 34009, 'COVID-19': 19970}


# Reading from COVID-QU-Ex-Dataset (No mask)

In [ ]:
for i,parent_directory in enumerate(os.listdir(COVID_QU_EX_PATH)):
    if not parent_directory.startswith('.'):
        full_parent_directory = os.path.join(COVID_QU_EX_PATH,parent_directory)
        for j,category in enumerate(os.listdir(full_parent_directory)):
            if not category.startswith('.'):
                full_category_directory = os.path.join(full_parent_directory,category)
                for k,img_dir in enumerate(os.listdir(full_category_directory)):
                    if not img_dir.startswith('.'):
                        count[category] += 1
                        entry = [int(count[category]) ,os.path.join('COVID-QU-Ex-dataset',
                                                                    'Lung Segmentation Data',
                                                                    'Lung Segmentation Data'
                                                                    ,parent_directory,
                                                                    category,
                                                                    'images',
                                                                    img_dir)
                        , category]
                        filename_label[category].append(entry)
print(count)

### Finally, writing to txt file

In [20]:
np.random.seed(seed=2)
test_per = 0.1
for key in filename_label.keys():
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    else:
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    for i,patient in enumerate(filename_label[key]):
        if int(patient[0]) in test_patients:
            test.append(patient)
            test_count[key] += 1
        else:
            train.append(patient)
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 7129, 'pneumonia': 3400, 'COVID-19': 1997}
train count:  {'normal': 64166, 'pneumonia': 30609, 'COVID-19': 17973}


In [21]:
WORK_DIR = os.path.dirname(os.path.realpath(__name__))
WORK_DIR = Path(WORK_DIR)
WORK_DIR = WORK_DIR.parent.absolute()
train_file = open(os.path.join(WORK_DIR,'split_files/alvaro_train_split.txt'),"w")
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open(os.path.join(WORK_DIR,'split_files/alvaro_test_split.txt'), "w")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()

# Building a Smaller DB for Testing

In [33]:
MAX_IMAGES_PER_CLASS = 1000

In [34]:
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
mapping = {0:'normal',1:'COVID-19',2:'pneumonia'}

### For MAC

In [37]:
for j,directory in enumerate(os.listdir(PAPER_IMAGES_PATH)):
    if not directory.startswith('.'):
        for i,file in enumerate(os.listdir(os.path.join(PAPER_IMAGES_PATH,directory))):
            try:
                cat = int(file.split('__')[1])
            except Exception as e:
                cat = int(file.split('_')[1])
            finally:
                if(count[mapping[cat]]<MAX_IMAGES_PER_CLASS):
                    count[mapping[cat]] += 1
                    entry = [int(count[mapping[cat]]),directory + '/' + file,mapping[cat]]
                    filename_label[mapping[cat]].append(entry)
print(count)

{'normal': 1000, 'pneumonia': 1000, 'COVID-19': 1000}


In [38]:
np.random.seed(seed=2)
test_per = 0.1
for key in filename_label.keys():
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    else:
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    for i,patient in enumerate(filename_label[key]):
        if int(patient[0]) in test_patients:
            test.append(patient)
            test_count[key] += 1
        else:
            train.append(patient)
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 100, 'pneumonia': 100, 'COVID-19': 100}
train count:  {'normal': 900, 'pneumonia': 900, 'COVID-19': 900}


In [39]:
WORK_DIR = os.path.dirname(os.path.realpath(__name__))
WORK_DIR = Path(WORK_DIR)
WORK_DIR = WORK_DIR.parent.absolute()

train_file = open(os.path.join(WORK_DIR,'split_files/ubuntu_train_split.txt'),"a+")
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open(os.path.join(WORK_DIR,'split_files/ubuntu_test_split.txt'), "a+")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()

# Full DB Building

In [ ]:
paths = ''